In [38]:
import torch
import torch.nn as nn
from torch.nn import functional as F
from torchvision import transforms
from torch.utils.data import DataLoader
from torchvision.datasets import MNIST
import pytorch_lightning as pl

In [39]:
transform = Compose(
    [
        ToTensor(),
    ]
)

In [40]:
# get the data
train = MNIST(
    root="./dara/",
    download=True,
    train=True,
    transform=transform
)
val = MNIST(
    root="./data/",
    download=True,
    train=False,
    transform=transform
)

In [90]:
class LinearModel(pl.LightningModule):

    def __init__(self):
        super().__init__()
        self.l1 = torch.nn.Linear(28 * 28, 10)

    def forward(self, x):
        return torch.relu(self.l1(x.view(x.size(0), -1)))

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = F.cross_entropy(y_hat, y)
        tensorboard_logs = {'train_loss': loss}
        return {'loss': loss, 'log': tensorboard_logs}

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=0.001)

    def train_dataloader(self):
        dataset = MNIST(os.getcwd(), train=True, download=True, transform=ToTensor())
        loader = DataLoader(dataset, batch_size=32, num_workers=4, shuffle=True)
        return loader
    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        return {'val_loss': F.cross_entropy(y_hat, y)}

    def validation_epoch_end(self, outputs):
        avg_loss = torch.stack([x['val_loss'] for x in outputs]).mean()
        tensorboard_logs = {'val_loss': avg_loss}
        return {'val_loss': avg_loss, 'log': tensorboard_logs}

    def val_dataloader(self):
        # TODO: do a real train/val split
        dataset = MNIST(os.getcwd(), train=False, download=True, transform=transforms.ToTensor())
        loader = DataLoader(dataset, batch_size=32, num_workers=4)
        return loader
    
    def test_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        return {'test_loss': F.cross_entropy(y_hat, y)}

    def test_epoch_end(self, outputs):
        avg_loss = torch.stack([x['test_loss'] for x in outputs]).mean()
        tensorboard_logs = {'test_loss': avg_loss}
        return {'avg_test_loss': avg_loss, 'log': tensorboard_logs}

    def test_dataloader(self):
        # TODO: do a real train/val split
        dataset = MNIST(os.getcwd(), train=False, download=True, transform=transforms.ToTensor())
        loader = DataLoader(dataset, batch_size=32, num_workers=4)
        return loader
    

In [91]:
model = LinearModel()

In [99]:
trainer = pl.Trainer(max_epochs=10, num_processes=1)

GPU available: True, used: False
No environment variable for node rank defined. Set as 0.


In [100]:
# trainer = pl.Trainer

In [ ]:
trainer.fit(model)


  | Name | Type   | Params
----------------------------
0 | l1   | Linear | 7 K   


Epoch 1:  86%|████████▌ | 1875/2188 [00:11<00:01, 157.57it/s, loss=0.989, v_num=11]
Validating: 0it [00:00, ?it/s]
Epoch 2:   0%|          | 0/2188 [00:00<?, ?it/s, loss=0.989, v_num=11]            

/home/macab/miniconda3/lib/python3.7/site-packages/pytorch_lightning/utilities/distributed.py:23: UserWarning: Did not find hyperparameters at model hparams. Saving checkpoint without hyperparameters.
  warnings.warn(*args, **kwargs)


Epoch 2:  86%|████████▌ | 1875/2188 [00:11<00:01, 164.92it/s, loss=0.945, v_num=11]
Validating: 0it [00:00, ?it/s]
Epoch 3:  86%|████████▌ | 1875/2188 [00:11<00:01, 167.44it/s, loss=0.990, v_num=11]
Validating: 0it [00:00, ?it/s]
Epoch 4:  86%|████████▌ | 1875/2188 [00:11<00:01, 162.24it/s, loss=0.995, v_num=11]
Validating: 0it [00:00, ?it/s]
Epoch 5:  86%|████████▌ | 1875/2188 [00:11<00:01, 170.42it/s, loss=0.938, v_num=11]
Validating: 0it [00:00, ?it/s]
Epoch 6:  72%|███████▏  | 1567/2188 [00:09<00:03, 167.47it/s, loss=0.973, v_num=11]

In [69]:
trainer.test()

Testing:  79%|███████▊  | 246/313 [00:00<00:02, 25.08it/s]--------------------------------------------------------------------------------
TEST RESULTS
{'avg_test_loss': tensor(1.1330, device='cuda:0'),
 'test_loss': tensor(1.1330, device='cuda:0')}
--------------------------------------------------------------------------------
Testing: 100%|██████████| 313/313 [00:00<00:00, 602.65it/s]


In [79]:
model.test_dataloader()